<a href="https://colab.research.google.com/github/snpushpi/Differential-Privacy-in-Split-Learning/blob/main/SplitNNRON_GaussV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import scipy

class RONGauss:
    """RON-Gauss: Synthesizing Data with Differential Privacy
    This module implements RON-Gauss, which is a method for non-interactive differentially-private data release based on 
    random orthornormal (RON) projection and Gaussian generative model. RON-Gauss leverages the Diaconis-Freedman-Meckes (DFM) effect,
    which states that most random projections of high-dimensional data approaches Gaussian.
    The main method of the `RONGauss` class is `_generate_dpdata()`. It takes in the original data as inputs, and, depending
    on the algorithm chosen, returns the differentially private data.
    
    Parameters
    ----------
    algorithm : string {'supervised', 'unsupervised', 'gmm'}
        supervised : 
            implements RON-Gauss for supervised learning, especially for regression. This algorithm
            requires both the feature data and the target label as inputs. In addition, it requires
            `max_y` to be specified.
        unsupervised :
            implements RON-Gauss for unsupervised learning, so only the feature data are required.
            This mode will return `None` for `dp_y`.
        gmm :
            implements RON-Gauss for classification. This algorithm requires both the feature data
            and the target label as inputs. The target label has to be categorical.
    epsilon_mean : float (default 1.0)
        The privacy budget for computing the mean. The sum of `epsilon_mean` and `epsilon_cov` is the total
        privacy budget spent.
    epsilon_cov : float (default 1.0)
        The privacy budget for computing the covariance. The sum of `epsilon_mean` and `epsilon_cov` is the total
        privacy budget spent.
    
    
    Examples
    -------
    >>> import numpy as np
    >>> X = np.random.normal(size=(1000,100))
    >>> dim = 10
    >>> # try unsupervised
    >>> rongauss_unsup = ron_gauss.RONGauss(algorithm='unsupervised')
    >>> dp_x, _ = rongauss_unsup.generate_dpdata(X, dim)
    >>> # try supervised
    >>> y = np.random.uniform(low=0.0, high=1.0, size=1000)
    >>> rongauss_sup = ron_gauss.RONGauss(algorithm='supervised')
    >>> dp_x, dp_y = rongauss_sup.generate_dpdata(X, dim, y, max_y = 1.0)
    >>> # try gmm
    >>> y = np.random.choice([0,1], size=1000)
    >>> rongauss_gmm = ron_gauss.RONGauss(algorithm='gmm')
    >>> dp_x, dp_y = rongauss_gmm.generate_dpdata(X, dim, y)
    """
    
    def __init__(self, algorithm="supervised", epsilon_mean=1.0, epsilon_cov=1.0):
        self.algorithm = algorithm
        self.epsilon_mean = epsilon_mean
        self.epsilon_cov = epsilon_cov

    def generate_dpdata(
        self,
        X,
        dimension,
        y=None,
        max_y=None,
        n_samples=None,
        reconstruct=True,
        centering=False,
        prng_seed=None,
    ):
        """Generate differentially-private dataset using RON-Gauss
        Parameters
        ----------
        X : numpy.ndarray, shape = [N_samples, M_features]
            Feature data.
        dimension : int < M_features
            The dimension for the data to be reduced to.
        y : numpy.ndarray, shape = [n_samples] (default None)
            Target values.
            unsupervised : this parameter is not used.
            supervised : required.
            gmm : required and the values should be categorical.
        n_samples : int (default None)
            The number of samples to be synthesized. If None is passed, the returned number of samples will
            be equal to N_samples of X.
        max_y : float (default None)
            The maximum absolute value that the target label can take. For example, if y is [0,1], then
            max_y = 1. If y is [-2,1], then max_y = 2. This is required and used by the supervised
            algorithm only.
        reconstruct : bool (default True)
            An option to reconstrut the projected synthesized data back to the original space. If True, the
            returned data will have the same dimension as X. If False, the returned data will have the dimension
            specified by the parameter `dimension`.
        centering : bool (default False)
            An option to automatically center the synthesized data. If False, the mean will be the
            differentially-private mean derived from X. This parameter is always False for 'gmm'.
        prng_seed : int (default None)
            This is to specify the seed used in randomized algorithms used.
        
        Returns
        -------
        x_dp : numpy.ndarray, shape = [n_samples, M_features] or [n_samples, dimensions]
            The differentially-private feature data. If `reconstruct` is True, this will be [n_samples, M_features].
            If `reconstruct` is False, it will be [n_samples, dimensions].
        y_dp : numpy.ndarray, shape = [n_samples]
            For `unsupervised`, this will be None.
            For `supervised` and `gmm`, this will be the differentially private target label.
        """
        (n, m) = X.shape
        if n_samples is None:
            n_samples = n

        if self.algorithm == "unsupervised":
            x_dp = self._unsupervised_rongauss(X, dimension, n_samples, reconstruct, centering, prng_seed)
            y_dp = None

        elif self.algorithm == "supervised":
            x_dp, y_dp = self._supervised_rongauss(X, dimension, y, n_samples, max_y, reconstruct, centering, prng_seed)

        elif self.algorithm == "gmm":
            x_dp, y_dp = self._gmm_rongauss(X,dimension, y, n_samples, reconstruct, prng_seed)
        
        return (x_dp, y_dp)
    
    def _unsupervised_rongauss(
        self,
        X,
        dimension,
        n_samples,
        reconstruct,
        centering,
        prng_seed,
    ):
        """Generate differentially-private dataset using the unsupervised RON-Gauss
        Parameters
        ----------
        X : numpy.ndarray, shape = [N_samples, M_features]
            Feature data.
        dimension : int < M_features
            The dimension for the data to be reduced to.
        n_samples : int (default None)
            The number of samples to be synthesized. If None is passed, the returned number of samples will
            be equal to N_samples of X.
        reconstruct : bool (default True)
            An option to reconstrut the projected synthesized data back to the original space. If True, the
            returned data will have the same dimension as X. If False, the returned data will have the dimension
            specified by the parameter `dimension`.
        centering : bool (default False)
            An option to automatically center the synthesized data. If False, the mean will be the
            differentially-private mean derived from X. This parameter is always False for 'gmm'.
        prng_seed : int (default None)
            This is to specify the seed used in randomized algorithms used.
        
        Returns
        -------
        x_dp : numpy.ndarray, shape = [n_samples, M_features] or [n_samples, dimensions]
            The differentially-private feature data. If `reconstruct` is True, this will be [n_samples, M_features].
            If `reconstruct` is False, it will be [n_samples, dimensions].
        """
        prng = np.random.RandomState(prng_seed)
        (x_bar, mu_dp) = self._data_preprocessing(X, self.epsilon_mean, prng)
        (x_tilda, proj_matrix) = self._apply_ron_projection(x_bar, dimension, prng)
        (n, p) = x_tilda.shape
        noise_var = (2.0 * np.sqrt(p)) / (n * self.epsilon_cov)
        cov_matrix = np.inner(x_tilda.T, x_tilda.T) / n
        laplace_noise = prng.laplace(scale=noise_var, size=(p, p))
        cov_dp = cov_matrix + laplace_noise
        synth_data = prng.multivariate_normal(np.zeros(p), cov_dp, n_samples)
        x_dp = synth_data
        if reconstruct:
            x_dp = self._reconstruction(x_dp, proj_matrix)
        else:
            #project the mean down to the lower dimention
            mu_dp = np.inner(mu_dp, proj_matrix)
        self._mu_dp = mu_dp

        if not centering:
            x_dp = x_dp + mu_dp
        return x_dp

    def _supervised_rongauss(
        self,
        X,
        dimension,
        y,
        n_samples,
        max_y,
        reconstruct,
        centering,
        prng_seed,
    ):  
        """Generate differentially-private dataset using the supervised RON-Gauss
        Parameters
        ----------
        X : numpy.ndarray, shape = [N_samples, M_features]
            Feature data.
        dimension : int < M_features
            The dimension for the data to be reduced to.
        y : numpy.ndarray, shape = [n_samples] (default None)
            Target values.
        n_samples : int (default None)
            The number of samples to be synthesized. If None is passed, the returned number of samples will
            be equal to N_samples of X.
        max_y : float
            The maximum absolute value that the target label can take. For example, if y is [0,1], then
            max_y = 1. If y is [-2,1], then max_y = 2.
        reconstruct : bool (default True)
            An option to reconstrut the projected synthesized data back to the original space. If True, the
            returned data will have the same dimension as X. If False, the returned data will have the dimension
            specified by the parameter `dimension`.
        centering : bool (default False)
            An option to automatically center the synthesized data. If False, the mean will be the
            differentially-private mean derived from X. This parameter is always False for 'gmm'.
        prng_seed : int (default None)
            This is to specify the seed used in randomized algorithms used.
        
        Returns
        -------
        x_dp : numpy.ndarray, shape = [n_samples, M_features] or [n_samples, dimensions]
            Differentially-private feature data. If `reconstruct` is True, this will be [n_samples, M_features].
            If `reconstruct` is False, it will be [n_samples, dimensions].
        y_dp : numpy.ndarray, shape = [n_samples]
            Differentially private target label.
        """

        prng = np.random.RandomState(prng_seed)
        (x_bar, mu_dp) = self._data_preprocessing(X, self.epsilon_mean, prng)
        (x_tilda, proj_matrix) = self._apply_ron_projection(x_bar, dimension, prng)

        (n, p) = x_tilda.shape
        noise_var = (2.0 * np.sqrt(p) + 4.0 * np.sqrt(p) * max_y + max_y ** 2) / (
            n * self.epsilon_cov
        )
        y_reshaped = y.reshape(len(y), 1)
        augmented_mat = np.hstack((x_tilda, y_reshaped))
        cov_matrix = np.inner(augmented_mat.T, augmented_mat.T) / n
        laplace_noise = prng.laplace(scale=noise_var, size=cov_matrix.shape)
        cov_dp = cov_matrix + laplace_noise

        synth_data = prng.multivariate_normal(np.zeros(p + 1), cov_dp, n_samples)
        x_dp = synth_data[:, 0:-1]
        y_dp = synth_data[:, -1]
        if reconstruct:
            x_dp = self._reconstruction(x_dp, proj_matrix)
        else:
            #project the mean down to the lower dimention
            mu_dp = np.inner(mu_dp, proj_matrix)
        self._mu_dp = mu_dp

        if not centering:
            x_dp = x_dp + mu_dp
        
        return (x_dp, y_dp)

    def _gmm_rongauss(
        self,
        X,
        dimension,
        y,
        n_samples,
        reconstruct,
        prng_seed,
    ):
        """Generate differentially-private dataset using the GMM RON-Gauss
        Parameters
        ----------
        X : numpy.ndarray, shape = [N_samples, M_features]
            Feature data.
        dimension : int < M_features
            The dimension for the data to be reduced to.
        y : numpy.ndarray, shape = [n_samples] (default None)
            Target values, which hould be categorical.
        n_samples : int (default None)
            The number of samples to be synthesized. If None is passed, the returned number of samples will
            be equal to N_samples of X.
        reconstruct : bool (default True)
            An option to reconstrut the projected synthesized data back to the original space. If True, the
            returned data will have the same dimension as X. If False, the returned data will have the dimension
            specified by the parameter `dimension`.
        centering : bool (default False)
            An option to automatically center the synthesized data. If False, the mean will be the
            differentially-private mean derived from X. This parameter is always False for 'gmm'.
        prng_seed : int (default None)
            This is to specify the seed used in randomized algorithms used.
        
        Returns
        -------
        x_dp : numpy.ndarray, shape = [n_samples, M_features] or [n_samples, dimensions]
            Differentially-private feature data. If `reconstruct` is True, this will be [n_samples, M_features].
            If `reconstruct` is False, it will be [n_samples, dimensions].
        y_dp : numpy.ndarray, shape = [n_samples]
            Differentially private target label.
        """
        prng = np.random.RandomState(prng_seed)
        syn_x = None
        syn_y = np.array([])
        for label in np.unique(y):
            idx = np.where(y == label)
            x_class = X[idx]
            (x_bar, mu_dp) = self._data_preprocessing(x_class, self.epsilon_mean, prng)
            (x_tilda, proj_matrix) = self._apply_ron_projection(x_bar, dimension, prng)

            (n, p) = x_tilda.shape
            noise_var = (2.0 * np.sqrt(p)) / (n * self.epsilon_cov)
            mu_dp_tilda = np.inner(mu_dp, proj_matrix)
            cov_matrix = np.inner(x_tilda.T, x_tilda.T) / n
            laplace_noise = prng.laplace(scale=noise_var, size=(p, p))
            cov_dp = cov_matrix + laplace_noise
            synth_data = prng.multivariate_normal(mu_dp_tilda, cov_dp, n_samples)

            if reconstruct:
                synth_data = self._reconstruction(synth_data, proj_matrix)
            if syn_x is None:
                syn_x = synth_data
            else:
                syn_x = np.vstack((syn_x, synth_data))
                
            syn_y = np.append(syn_y, label * np.ones(n_samples))
        return syn_x, syn_y

    @staticmethod
    def _data_preprocessing(X, epsilon_mean, prng=None):
        """
        This is the DATA_PREPROCESSING algorithm based on Algo. 1 in the paper.
        Parameters
        ----------
        X : numpy.ndarray, shape = [N_samples, M_features]
            Feature data.
        epsilon_mean : float
            The privacy budget used for computing the mean.
        prng_seed : int (default None)
            This is to specify the seed used in randomized algorithms used.
        
        Returns
        -------
        x_bar : numpy.ndarray, shape = [N_samples, M_features]
            The pre-processed data which are pre-normalized, centered, and re-normalized.
        mean_dp : numpy.ndarray, shape = [M_features]
            The differentially-private mean used in the centering.
        """
        if prng is None:
            prng = np.random.RandomState()
        (n, m) = X.shape
        # pre-normalize
        x_norm = RONGauss._normalize_sample_wise(X)
        # derive dp-mean
        mu = np.mean(x_norm, axis=0)
        noise_var_mu = np.sqrt(m) / (n * epsilon_mean)
        laplace_noise = prng.laplace(scale=noise_var_mu, size=m)
        mean_dp = mu + laplace_noise
        # centering
        x_bar = x_norm - mean_dp
        # re-normalize
        x_bar = RONGauss._normalize_sample_wise(x_bar)
        return x_bar, mean_dp

    def _apply_ron_projection(self, x_bar, dimension, prng=None):
        """
        This is the RON_PROJECTION algorithm based on Algo. 2 in the paper.
        Parameters
        ----------
        x_bar : numpy.ndarray, shape = [N_samples, M_features]
            Feature data.
        dimension : int < M_features
            The dimension for the data to be reduced to.
        prng_seed : int (default None)
            This is to specify the seed used in randomized algorithms used.
        
        Returns
        -------
        x_tilda : numpy.ndarray, shape = [N_samples, dimension]
            The dimension-reduced data.
        ron_matrix : numpy.ndarray, shape = [dimension, M_features]
            The RON projection matrix used for dimensionality reduction.
        """
        (n, m) = x_bar.shape
        full_projection_matrix = self._generate_ron_matrix(m, prng)
        ron_matrix = full_projection_matrix[0:dimension]  # take the rows
        x_tilda = np.inner(x_bar, ron_matrix)
        return x_tilda, ron_matrix

    def _reconstruction(self, x_projected, ron_matrix):
        """
        The function used to project the dimension-reduced data back to the original space.
        Parameters
        ----------
        x_projected : numpy.ndarray, shape = [N_samples, P_dimension]
            The dimension-reduced data.
        ron_matrix : numpy.ndarray, shpae = [P_dimension, M_features]
            The RON projection matrix used to produce the dimension-reduced data.
        
        Returns
        -------
        x_reconstructed : numpy.ndarray, shape = [N_samples, M_features]
            The reconstructed data.
        """
        x_reconstructed = np.inner(x_projected, ron_matrix.T)
        return x_reconstructed

    def _generate_ron_matrix(self, m, prng=None):
        """
        Generate a RON projection matrix using QR factorization.
        Parameters
        ----------
        m : int
            The dimension of the projection matrix.
        prng_seed : int (default None)
            This is to specify the seed used in randomized algorithms used.
        
        Returns
        -------
        ron_matrix : numpy.ndarray, shape = [m, m]
            The RON projection matrix.
        """
        if prng is None:
            prng = np.random.RandomState()
        # generate random matrix
        random_matrix = prng.uniform(size=(m, m))
        # QR factorization
        q_matrix, r_matrix = scipy.linalg.qr(random_matrix) #numpy.linalg.qr(random_matrix, mode='complete')
        ron_matrix = q_matrix
        return ron_matrix
        
    @staticmethod
    def _normalize_sample_wise(x):
        """
        Sample-wise normalization
        Parameters
        ----------
        x : numpy.ndarray, shape = [N_samples, M_features]
            Feature data.
        
        Returns
        -------
        x_normalized : numpy.ndarray, shape = [N_samples, M_features]
            The sample-wise normalized data
        """
        (n,p) = x.shape
        sample_norms = np.linalg.norm(x, axis=1) #norms of each sample
        x_normalized = x/(np.outer(sample_norms,np.ones(p)))
        return x_normalized

time: 349 ms


In [9]:
import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import sys
import numpy as np 
import copy

! pip install ipython-autotime
%load_ext autotime

! pip install ipython-autotime
%load_ext autotime
print(f"Python: {sys.version}")
print(f"Pytorch: {torch.__version__}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  print(f'GPU: {torch.cuda.current_device()}, {torch.cuda.device_count()}, {torch.cuda.get_device_name(0)}, {torch.cuda.is_available()}')
else: print(f'Device: cpu')

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Python: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
Pytorch: 1.7.0+cu101
Device: cpu
time: 5.48 s


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 35.7 s


In [4]:
mnist_data_path = '/content/drive/My Drive/archive'

time: 1.01 ms


In [10]:
transform = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))])

# load training set 
mnist_trainset = torchvision.datasets.MNIST(mnist_data_path, train=True, transform=transform, download=True)
mnist_train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, shuffle=True)

# load test set
mnist_testset = torchvision.datasets.MNIST(mnist_data_path, train=False, transform=transform, download=True)
mnist_test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=128, shuffle=True)

from torch.utils.data.sampler import SubsetRandomSampler

total_size = len(mnist_trainset)

split1 = total_size // 4
split2 = split1*2
split3 = split1*3

indices = list(range(total_size))

alice_idx = indices[:split1]
bob_idx = indices[split1:split2]
mike_idx = indices[split2:split3]
rose_idc = indices[split3:]

alice_sampler = SubsetRandomSampler(alice_idx)
bob_sampler = SubsetRandomSampler(bob_idx)
mike_sampler = SubsetRandomSampler(mike_idx)
rose_sampler = SubsetRandomSampler(rose_idc)


alice_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=alice_sampler)
bob_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=bob_sampler)
mike_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=mike_sampler)
rose_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, sampler=rose_sampler)

data_loaders = [alice_loader, bob_loader, mike_loader, rose_loader ]

print(f'Data at alice: {len(alice_sampler)} \t Batches: {len(alice_loader)}')
print(f'Data at bob: {len(bob_sampler)} \t Batches: {len(alice_loader)}')
print(f'Data at mike: {len(mike_sampler)} \t Batches: {len(mike_loader)}')
print(f'Data at rose: {len(rose_sampler)} \t Batches: {len(rose_loader)}')

Data at alice: 15000 	 Batches: 118
Data at bob: 15000 	 Batches: 118
Data at mike: 15000 	 Batches: 118
Data at rose: 15000 	 Batches: 118
time: 150 ms


In [11]:
client_model = torch.nn.Sequential(
                torch.nn.Conv2d(1, 32, kernel_size=5, padding=0, stride=1),  
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2),
                torch.nn.Conv2d(32, 32, kernel_size=5, padding=0, stride=1),  
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2),
                )

# send a copy to each data holder
alice_model = copy.deepcopy(client_model)
bob_model = copy.deepcopy(client_model)
mike_model = copy.deepcopy(client_model)
rose_model = copy.deepcopy(client_model)

# keep server copy at Server
server_model = torch.nn.Sequential(
                torch.nn.Flatten(),
                torch.nn.Linear(512, 256),
                torch.nn.ReLU(),
                torch.nn.Linear(256, 10),  
                # torch.nn.Softmax(dim=-1),
        )


models = [alice_model, bob_model, mike_model, rose_model]

# Create optimizers for each model
optimizers = [optim.Adam(model.parameters(), lr=0.01) for model in models]
#def cor_loss(x, split):
#  retun value

#def total_loss(cor_loss, celoss):
#  return x*cor_loss + y*celoss

time: 26.9 ms


In [12]:
server_optimizer = optim.Adam(server_model.parameters(), lr=0.01)
server_loss = nn.CrossEntropyLoss()

time: 1.81 ms


In [ ]:
epochs = 7
dim = 600

for e in range(epochs):
  running_loss = 0

  # iterate based on batch numbers assuming it is unified acorss all clients
  # a more efficient solution is to assign the batch size at each client relative on its data size
  # this would guarantee less latency at the server side given that client-side training is parallaized 
  for l in range(1):
    for opt in optimizers:
      opt.zero_grad()
    
    server_optimizer.zero_grad()

    lst_of_vars = []

    for i in range(len(data_loaders)): 
      imgs, lbls = next(iter(data_loaders[i]))
      imgs = imgs.reshape((128,784)).numpy()
      lbls = lbls.reshape((128,)).numpy()
      rongauss_gmm = RONGauss(algorithm='gmm') 
      dp_imgs, dp_lbls = rongauss_gmm.generate_dpdata(imgs, dim, lbls)
      dp_imgs = torch.from_numpy(dp_imgs)
      dp_lbls = torch.from_numpy(dp_lbls)
      imgs = torch.reshape(dp_imgs,(1280,1,28,28))
      lbls = torch.reshape(dp_lbls,(1280,))
      lbls = lbls.type(torch.LongTensor)
      split_output = models[i](imgs.float()) 
      split_layer_output = split_output.clone().detach().requires_grad_(True)
      #cor_loss = (imgs, split_layer_output)
      server_output = server_model(split_layer_output)
      
      loss = server_loss(server_output, lbls)
      #total = toal()

      lst_of_vars.append({'split_output': split_output, 'split_layer_output': split_layer_output, 'loss':loss})

    loss = 0  
    for i in range(len(data_loaders)):
      loss += lst_of_vars[i]['loss']

    avg_loss = loss / len(data_loaders)
    running_loss += avg_loss

    avg_loss.backward()

    for i in range(len(data_loaders)):
      split_gradients = lst_of_vars[i]['split_layer_output'].grad.clone().detach()
      lst_of_vars[i]['split_output'].backward(split_gradients)
    
    server_optimizer.step()
    
    for opt in optimizers:
      opt.step()

  print("Epoch {} - Training loss: {}".format(e+1, running_loss/len(data_loaders[0])))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:305: RuntimeWarning: covariance is not positive-semidefinite.


Epoch 1 - Training loss: 0.019513459876179695
Epoch 2 - Training loss: 0.01951332949101925


In [51]:
def test_transform(imgs):
    imgs = imgs.numpy()
    m,n,p,q = imgs.shape
    imgs = imgs.reshape((m*n,p*q))
    lbls = np.zeros((m*n,))
    rongauss_gmm = RONGauss(algorithm='gmm') 
    dp_imgs, dp_lbls = rongauss_gmm.generate_dpdata(imgs, dim, lbls)
    dp_imgs = torch.from_numpy(dp_imgs)
    dp_lbls = torch.from_numpy(dp_lbls)
    imgs = torch.reshape(dp_imgs,(m,n,p,q))
    return imgs

time: 4.11 ms


In [53]:
correct = 0
total = 0
with torch.no_grad():
    for data in mnist_test_loader:
        images, labels = data 
        print('hi')
        images = test_transform(images)
        outputs = bob_model(images.float())
        outputs = server_model(outputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the 10000 test images using SPLIT INFERENCE: %d %%' % (100 * correct / total))

hi


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:305: RuntimeWarning: covariance is not positive-semidefinite.


hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Accuracy on the 10000 test images using SPLIT INFERENCE: 10 %
time: 37.5 s
